In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras

# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units, n_feature):
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input), name="root_word_input")
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    print(state_h.shape)
    # define training decoder
    decoder_inputs = Input(shape=(None, n_output), name="target_word_input")
    
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    
    feature_input = Input(shape=(None, n_feature), name="word_feature_input")
    feat_out = Dense(10, activation="relu", name="feature_output")(feature_input)
    print(feat_out.shape)
    x = Concatenate(name="feature_merge")([decoder_outputs, feat_out])
    decoder_dense = Dense(n_output, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(x)
    
    model = Model([encoder_inputs, decoder_inputs, feature_input], decoder_outputs)
#     define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    
    word_feature_input = Input(shape=(None, n_feature), name="word_feature_input")
    word_feat_out = Dense(10, activation="relu")(word_feature_input)
    word_out = Concatenate()([decoder_outputs, word_feat_out])
    
    decoder_outputs = decoder_dense(word_out)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs + [word_feature_input] , [decoder_outputs] + decoder_states)

    return model, encoder_model, decoder_model


c:\users\amany\appdata\local\conda\conda\envs\gputf3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
def seq(n_input, n_output, n_units, n_feature):
    # define training encoder
    feat_units = 15
    encoder_inputs = Input(shape=(None, n_input), name="root_word_input")
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    
    feature_input = Input(shape=(n_feature,), name="word_feature_input")
    feat_out = Dense(feat_units, activation="relu", name="feature_output")(feature_input)
    x = Concatenate()([state_h, feat_out])
    x2 = Concatenate()([state_c, feat_out])
    state_h = Dense(n_units, activation='relu')(x)
    state_c = Dense(n_units, activation='relu')(x2)
    encoder_states = [state_h, state_c]
    
    decoder_inputs = Input(shape=(None, n_output), name="target_word_input")
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
#     decoder_outputs, _, _ = decoder_lstm(decoder_inputs)
    decoder_dense = Dense(n_output, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([encoder_inputs, decoder_inputs, feature_input], decoder_outputs)
#     define inference encoder
    encoder_model = Model([encoder_inputs, feature_input], encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    return model, encoder_model, decoder_model

In [5]:
def seq2(n_input, n_output, n_feature, n_enc_units, n_dec_units):
    # define training encoder
    feat_units = 15
    encoder_inputs = Input(shape=(None, n_input), name="root_word_input")
    encoder = LSTM(n_enc_units, return_state=True, name="encoder_lstm")
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    
    feature_input = Input(shape=(n_feature,), name="word_feature_input")
    feat_out = Dense(feat_units, activation="relu", name="feature_output")(feature_input)
    x = Concatenate()([state_h, feat_out])
    x2 = Concatenate()([state_c, feat_out])
    state_h = Dense(n_dec_units, activation='sigmoid')(x)
    state_c = Dense(n_dec_units, activation='sigmoid')(x2)
    encoder_states = [state_h, state_c]
    
    decoder_inputs = Input(shape=(None, n_output), name="target_word_input")
    decoder_lstm = LSTM(n_dec_units, return_sequences=True, return_state=True, name="decoder_lstm")
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([encoder_inputs, decoder_inputs, feature_input], decoder_outputs)

    encoder_model = Model([encoder_inputs, feature_input], encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_dec_units,))
    decoder_state_input_c = Input(shape=(n_dec_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    return model, encoder_model, decoder_model

In [9]:
def conv_model(n_input, n_output, n_feature, n_enc_units, n_dec_units, feat_units = 10):
    root_word_input = Input(shape=(15, 28, 1), name="root_word_input")
    feature_input = Input(shape=(n_feature,), name="word_feature_input")
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(root_word_input)
    feat_out = Dense(feat_units, activation="relu", name="feature_output")(feature_input)
    x = MaxPooling2D(2, 2)(x)
    flat_output = Flatten()(x)
    x = Dense(100, activation='relu')(flat_output)
    x = Dropout(.4)(x)
    x = Concatenate()([x, feat_out])
#     state_h = Dense(n_dec_units, activation='relu')(x)
    state_h = Dense(n_dec_units, activation='relu')(x)
    state_h = Dropout(.4)(state_h)
    decoder_inputs = Input(shape=(None, n_output), name="target_word_input")
    decoder_gru = GRU(n_dec_units, return_sequences=True, return_state=True, name="decoder_gru")
    decoder_outputs, _= decoder_gru(decoder_inputs, initial_state=state_h)
    
    decoder_dense = Dense(n_output, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([root_word_input, decoder_inputs, feature_input], decoder_outputs)
    encoder_model = Model([root_word_input, feature_input], state_h)
    
    decoder_state_input_h = Input(shape=(n_dec_units,))
    decoder_outputs, state_h= decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs, decoder_state_input_h], [decoder_outputs, state_h])

    return model, encoder_model, decoder_model


In [3]:
def conv_model2(n_input, n_output, n_feature, n_enc_units, n_dec_units, feat_units = 10):
    root_word_input = Input(shape=(15, 28, 1), name="root_word_input")
    feature_input = Input(shape=(n_feature,), name="word_feature_input")
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(root_word_input)
    feat_out = Dense(feat_units, activation="relu", name="feature_output")(feature_input)
    x = MaxPooling2D(2, 2)(x)
    flat_output = Flatten()(x)
#     x = Dense(100, activation='relu')(flat_output)
    x = Dense(n_dec_units - feat_units, activation='relu')(flat_output)
#     state_h = Dropout(.4)(state_h)
    
    state_h = Concatenate()([x, feat_out])
    decoder_inputs = Input(shape=(None, n_output), name="target_word_input")
    decoder_gru = GRU(n_dec_units, return_sequences=True, return_state=True, name="decoder_gru")
    decoder_outputs, _= decoder_gru(decoder_inputs, initial_state=state_h)
    
    decoder_dense = Dense(n_output, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([root_word_input, decoder_inputs, feature_input], decoder_outputs)
    encoder_model = Model([root_word_input, feature_input], state_h)
    
    decoder_state_input_h = Input(shape=(n_dec_units,))
    decoder_outputs, state_h= decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs, decoder_state_input_h], [decoder_outputs, state_h])

    return model, encoder_model, decoder_model

In [6]:
def conv_model3(n_input, n_output, n_feature, n_enc_units, n_dec_units, feat_units = 10):
    root_word_input = Input(shape=(15, 28, 1), name="root_word_input")
    feature_input = Input(shape=(n_feature,), name="word_feature_input")
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(root_word_input)
    feat_out = Dense(feat_units, activation="relu", name="feature_output")(feature_input)
    x = MaxPooling2D(2, 2)(x)
    flat_output = Flatten()(x)
#     x = Dense(100, activation='relu')(flat_output)
    x = Dense(n_dec_units - feat_units, activation='relu')(flat_output)
#     state_h = Dropout(.4)(state_h)
    
    state_h = Concatenate()([x, feat_out])
    decoder_inputs = Input(shape=(None, n_output), name="target_word_input")
    x_decode, _ = GRU(n_dec_units, initial_state=state_h, return_sequences=True, return_state=True, name="decoder_gru_1")(decoder_inputs)
    decoder_gru = GRU(n_dec_units, return_sequences=True, return_state=True, name="decoder_gru")
    decoder_outputs, _= decoder_gru(x_decode, initial_state=state_h)
    
    decoder_dense = Dense(n_output, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([root_word_input, decoder_inputs, feature_input], decoder_outputs)
    encoder_model = Model([root_word_input, feature_input], state_h)
    
    decoder_state_input_h = Input(shape=(n_dec_units,))
    x_decode = GRU(n_dec_units, initial_state=decoder_state_input_h, name="decoder_gru_1")(decoder_inputs)
    decoder_outputs, state_h= decoder_gru(x_decode, initial_state=decoder_state_input_h)
    decoder_outputs, state_h= decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs, decoder_state_input_h], [decoder_outputs, state_h])

    return model, encoder_model, decoder_model

In [7]:
# train, infenc, infdec = conv_model3(27, 27, 29, 64,128)
# train.summary()

TypeError: ('Keyword argument not understood:', 'initial_state')

In [ ]:
def predict(infenc, infdec, source, feat, n_steps, cardinality):
    # encode
    state = infenc.predict([source, feat])
    # start of sequence input
    start = [0.0 for _ in range(cardinality)]
#     start[0] = 1
    target_seq = np.array(start).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return np.array(output)

def predict2(infenc, infdec, source, feat, n_steps, cardinality):
    # encode
    state = infenc.predict([source, feat])
    # start of sequence input
    start = [0.0 for _ in range(cardinality)]
#     start[0] = 1
    target_seq = np.array(start).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h= infdec.predict([target_seq, state])
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = h
        # update target sequence
        target_seq = yhat
    return np.array(output)